In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:
fingerprint_df = pd.read_csv('rssi.csv')
current_rssi_df = pd.read_csv('rssi_now.csv')


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# Load historical data from rssi.csv
historical_data = pd.read_csv('rssi.csv')

# Split historical data into MAC addresses and features (X) and target (y)
X = historical_data.drop(['MAC', 'RP'], axis=1)
y = historical_data['RP']

# Split historical data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features using standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train the K-Nearest Neighbors classifier using historical data
k = 3  # Number of neighbors
classifier = KNeighborsClassifier(n_neighbors=k)
classifier.fit(X_train_scaled, y_train)

# Load current data from rssi_now.csv
current_data = pd.read_csv('rssi_now.csv')

# Preprocess the current data
current_data['RSSI'] = current_data['RSSI'].str.extract('(-?\d+)').astype(float)

# Split current data into MAC addresses and features (X)
current_X = current_data.drop('MAC', axis=1)

# Scale the current RSSI values using the same scaler as the historical data
current_X_scaled = scaler.transform(current_X)

# Predict the room positions for the current RSSI values
predicted_positions = classifier.predict(current_X_scaled)

# Print the predicted positions
for position in predicted_positions:
    print("Predicted Position:", position)


Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)
Predicted Position: (-1.346,0.122)


In [ ]:
import pandas as pd
import re
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# Load historical data from rssi.csv
historical_data = pd.read_csv('rssi.csv')

# Load current RSSI values of the robot from rssi_now.csv
current_data = pd.read_csv('rssi_now.csv')

# Split historical data into MAC addresses and features (X) and target (y)
X = historical_data.drop(['MAC', 'RP'], axis=1)
y = historical_data['RP']

# Preprocess current RSSI values
current_data = current_data.applymap(lambda x: re.sub(r'\D', '', str(x))).astype(float)

# Scale the features using standardization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_scaled, y)

# Scale the current RSSI values using the same scaler as the historical data
current_X = current_data.drop('MAC', axis=1)
current_X_scaled = scaler.transform(current_X)

# Predict the room position for the current RSSI values
predicted_positions = knn.predict(current_X_scaled)

# Print the predicted room positions
print("Predicted Room Positions:")
for position in predicted_positions:
    print(position)


In [3]:
import pandas as pd
import re
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix

# Load historical data from rssi.csv
historical_data = pd.read_csv('rssi.csv')

# Load current RSSI values of the robot from rssi_now.csv
current_data = pd.read_csv('rssi_now.csv')

# Split historical data into MAC addresses and features (X) and target (y)
X = historical_data.drop(['MAC', 'RP'], axis=1)
y = historical_data['RP']

# Preprocess current RSSI values
current_data = current_data.applymap(lambda x: re.sub(r'\D', '', str(x))).astype(float)

# Scale the features using standardization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train the KNN classifier
k = 2  # Number of nearest neighbors to consider
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_scaled, y)

# Scale the current RSSI values using the same scaler as the historical data
current_X = current_data.drop('MAC', axis=1)
current_X_scaled = scaler.transform(current_X)

# Find the k nearest neighbors for the current RSSI values
distances, indices = knn.kneighbors(current_X_scaled)

# Calculate weights based on distances
weights = 1 / distances

# Sum the weights for each class
class_weights = {}
for i in range(len(indices)):
    for j in range(k):
        neighbor_index = indices[i][j]
        neighbor_class = y.iloc[neighbor_index]
        if neighbor_class in class_weights:
            class_weights[neighbor_class] += weights[i][j]
        else:
            class_weights[neighbor_class] = weights[i][j]

# Predict the room position for the current RSSI values based on weighted votes
predicted_positions = []
for i in range(len(indices)):
    class_votes = {}
    for j in range(k):
        neighbor_index = indices[i][j]
        neighbor_class = y.iloc[neighbor_index]
        if neighbor_class in class_votes:
            class_votes[neighbor_class] += class_weights[neighbor_class]
        else:
            class_votes[neighbor_class] = class_weights[neighbor_class]
    predicted_positions.append(max(class_votes, key=class_votes.get))

# Print the predicted room positions
print("Predicted Room Positions:")
for position in predicted_positions:
    print(position)

Predicted Room Positions:
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
(-1.346,0.122)
